In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [6]:
trash_df = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')

In [8]:
trash_df.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


In [14]:
trash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
 11  LONGITUDE         20198 non-null  float64
 12  LATITUDE          20198 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 2.0+ MB


In [22]:
multicounts = trash_df.groupby(['Incident Address'])['Date Opened'].count()

In [28]:
multicounts = multicounts.reset_index()

In [30]:
multicounts

,Incident Address,Date Opened
0,", Nashville, TN , United States",1
1,1 BELLE FORREST AVE C,1
2,10 Belle Forrest Ave,1
3,"100 Bluefield Square, Nashville, TN 37214, Uni...",1
4,100 Braxton Hill Ct,1
...,...,...
14116,compton ave & 15th ave s,1
14117,fransworth dr,1
14118,s 13th st & woodland st,1
14119,trimble rd & lindawood dr,1


In [34]:
two_or_more = multicounts[multicounts['Date Opened'] > 1]

In [36]:
two_or_more

,Incident Address,Date Opened
6,"100 Clydelan Ct, Nashville, TN 37205, United S...",2
11,"100 Lincoln Ct, Nashville, TN 37205, United St...",2
12,"100 Marshall Ct, Nashville, TN 37212, United S...",2
14,"100 Mountainhigh Dr, Nashville, TN 37013, Unit...",2
15,"100 Nashboro Greens, Nashville, TN 37217, Unit...",2
...,...,...
14084,"E Campbell Rd, Nashville, TN 37115, United States",2
14087,"Farmhill Ct, Nashville, TN 37013, United States",3
14091,"Granbery Park Dr, Nashville, TN 37027, United ...",7
14111,"Trevino Pl, Nashville, TN 37013, United States",2


In [60]:
two_or_more.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3298 entries, 6 to 14112
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Incident Address  3298 non-null   object
 1   Date Opened       3298 non-null   int64 
 2   Cost              3298 non-null   object
dtypes: int64(1), object(2)
memory usage: 103.1+ KB


In [72]:
def costs(row):
    return row * 200

In [74]:
missed_cost = two_or_more.apply(costs, axis=1)
print(missed_cost)

                                        Incident Address  Date Opened  \
6      100 Clydelan Ct, Nashville, TN 37205, United S...          400   
11     100 Lincoln Ct, Nashville, TN 37205, United St...          400   
12     100 Marshall Ct, Nashville, TN 37212, United S...          400   
14     100 Mountainhigh Dr, Nashville, TN 37013, Unit...          400   
15     100 Nashboro Greens, Nashville, TN 37217, Unit...          400   
...                                                  ...          ...   
14084  E Campbell Rd, Nashville, TN 37115, United Sta...          400   
14087  Farmhill Ct, Nashville, TN 37013, United State...          600   
14091  Granbery Park Dr, Nashville, TN 37027, United ...         1400   
14111  Trevino Pl, Nashville, TN 37013, United States...          400   
14112  Tusculum Rd, Nashville, TN , United StatesTusc...          600   

                                                    Cost  
6      11/1/201711/1/201711/1/201711/1/201711/1/20171...  
11   

In [76]:
missed_cost.reset_index()

,index,Incident Address,Date Opened,Cost
0,6,"100 Clydelan Ct, Nashville, TN 37205, United S...",400,11/1/201711/1/201711/1/201711/1/201711/1/20171...
1,11,"100 Lincoln Ct, Nashville, TN 37205, United St...",400,11/1/201711/1/201711/1/201711/1/201711/1/20171...
2,12,"100 Marshall Ct, Nashville, TN 37212, United S...",400,11/1/201711/1/201711/1/201711/1/201711/1/20171...
3,14,"100 Mountainhigh Dr, Nashville, TN 37013, Unit...",400,11/1/201711/1/201711/1/201711/1/201711/1/20171...
4,15,"100 Nashboro Greens, Nashville, TN 37217, Unit...",400,11/1/201711/1/201711/1/201711/1/201711/1/20171...
...,...,...,...,...
3293,14084,"E Campbell Rd, Nashville, TN 37115, United Sta...",400,6/20/20196/20/20196/20/20196/20/20196/20/20196...
3294,14087,"Farmhill Ct, Nashville, TN 37013, United State...",600,6/20/20196/20/20196/20/20196/20/20196/20/20196...
3295,14091,"Granbery Park Dr, Nashville, TN 37027, United ...",1400,6/20/20196/20/20196/20/20196/20/20196/20/20196...
3296,14111,"Trevino Pl, Nashville, TN 37013, United States...",400,6/20/20196/20/20196/20/20196/20/20196/20/20196...


In [80]:
missed_cost['Date Opened'].sum()

1878800